# Deep Neural networks for decoding Motor Imagery movements from EEG signals of patients with stroke

Stroke patients

- One hot encoding
- Split dataset (training, validation, test)
- StandardScaler() (Normalizzazione, standardizzazione)


Reti che si useremo: CNN, RNN, RRN-CNN, AE(?)
- se si lavora con le CNN aggiungere una dimensione al tensore creato (definendo la profondità)

- Creazione del modello
- Training (Cross-validation)
  - Hyperparameter tuning
    - learning rate
![](https://www.google.com/url?sa=i&url=https%3A%2F%2Fscikit-learn.org%2Fstable%2Fmodules%2Fcross_validation.html&psig=AOvVaw1CUIUTwyS-Ll2HwxJh8Jnq&ust=1713608199307000&source=images&cd=vfe&opi=89978449&ved=0CBIQjRxqFwoTCOD3v5GGzoUDFQAAAAAdAAAAABAJ)


- Performance


### Commenti
- Non lo se in questo caso conviene lavorare con i dataframe
- potremmo anche provare con i **segnali raw**!!!!!!
- Vedere le reti neurali che sono state utilizzate per Motor imagery
- Potremmo fare Data Augmentation
  - Utilizando un VAE o GAN
  - Aggiungendo rumore
- Di solito le prestazioni sono per un soggetto perchè il training viene fatto per ogni paziente (in quanto l'applicazione futura è per le BCI). Dopo averle calcolate si fa la media
  - soggetto-specifico!!!!
- In questo caso non possiamo lavorare per soggetto-specifico perchè significa che dovremmo dovremmo dividere 40 trials in training, validation e test e sarebbero troppo pochi!

# DATASET SPLITTING:
E' FONDAMENTALE mantenere le stesse proporzioni in training, validation e test set.

---


Un'idea potrebbe essere:
1. Mantenere lo stesso rapporto di **M e F**;
2. Mantenere lo stesso rapporto in base all**'età,** dopo aver stratificato in base al sesso;
3.Mantenere stesso rapporto in base al **lato della paralisi**, dopo aver suddiviso il dataset in base a sesso ed età.

In [21]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.io
import pandas as pd

import os

!pip install mne
import mne

from google.colab import drive
drive.mount('/content/drive')

import sys
sys.path.append('/content/drive/MyDrive/utils')
from df_utils import inspection_dataframe

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# @title install dataset ALESSIO
# How to use glob in Colab
#print(os.listdir('MyDrive')), NO, it is not correct

print(os.listdir('/content/drive/MyDrive/-Shared codes/PROJECT/Stroke dataset/edffile')) # Ale

['sub-22', 'sub-49', 'sub-14', 'sub-23', 'sub-40', 'sub-24', 'sub-25', 'sub-47', 'sub-13', 'sub-15', 'sub-01', 'sub-37', 'sub-48', 'sub-46', 'sub-12', 'sub-06', 'sub-41', 'sub-39', 'sub-08', 'sub-30', 'sub-28', 'sub-07', 'sub-09', 'sub-36', 'sub-31', 'sub-10', 'sub-44', 'sub-17', 'sub-43', 'sub-38', 'sub-11', 'sub-19', 'sub-45', 'sub-42', 'sub-26', 'sub-16', 'sub-27', 'sub-21', 'sub-18', 'sub-20', 'sub-04', 'sub-05', 'sub-32', 'sub-34', 'sub-33', 'sub-35', 'sub-03', 'sub-50', 'sub-29', 'sub-02']


# Data inspection
Utilizzando la libreria mne è possibile leggere i dati edf

- IN QUESTO CASO HO CARICATO SOLO UN SOGGETTO, DOBBIAMO CREARE UN CICLO FOR

In [ ]:
data=mne.io.read_raw_edf('/content/drive/MyDrive/-Shared codes/PROJECT/Stroke dataset/edffile/sub-01/eeg/sub-01_task-motor-imagery_eeg.edf')
data

Extracting EDF parameters from /content/drive/MyDrive/-Shared codes/PROJECT/Stroke dataset/edffile/sub-01/eeg/sub-01_task-motor-imagery_eeg.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...


<ipython-input-12-0af41bdfb17b>:1: RuntimeWarning: Physical range is not defined in following channels:
CPz
  data=mne.io.read_raw_edf('/content/drive/MyDrive/-Shared codes/PROJECT/Stroke dataset/edffile/sub-01/eeg/sub-01_task-motor-imagery_eeg.edf')


Measurement date,"December 01, 2022 18:58:12 GMT"
Experimenter,Unknown
Participant,X
Digitized points,Not available
Good channels,33 EEG
Bad channels,None
EOG channels,Not available
ECG channels,Not available
Sampling frequency,500.00 Hz
Highpass,0.00 Hz
Lowpass,250.00 Hz


In [ ]:
# Inspection of the data
EEG=data.get_data()
print(EEG) # array
Fs = data.info['sfreq']
print(f'Sampling frequency= {Fs} Hz')
channel_labels = data.info['ch_names']
print(f'Channels:{channel_labels}')

[[ 1.09646199e-04  1.08225590e-04  1.07014219e-04 ...  3.33184576e-06
   2.31319260e-06  1.25048957e-06]
 [ 3.32744281e-05  3.24673972e-05  3.16822766e-05 ... -2.66583468e-06
  -1.99026583e-06 -1.25261768e-06]
 [ 2.61710864e-05  2.52741216e-05  2.43840832e-05 ... -8.20974037e-07
  -7.91536968e-07 -7.41320792e-07]
 ...
 [ 4.14912743e-05  4.23754000e-05  4.32299138e-05 ... -1.36978350e-06
  -1.24287551e-06 -1.24710578e-06]
 [ 5.08534100e-05  5.00452557e-05  4.89232550e-05 ... -4.54831545e-06
  -4.47377694e-06 -4.70523862e-06]
 [ 1.00000000e-06  0.00000000e+00  0.00000000e+00 ...  0.00000000e+00
   0.00000000e+00  0.00000000e+00]]
Sampling frequency= 500.0 Hz


Ocular artifacts were monitored by horizontal (HEOR, HEOL) bipolar EOG electrodes.

In [ ]:
FP1 = data.get_data(picks='FP1')
time=data.info['time']

Fs=500 #Hz
Ts=1/Fs

#t=np.arange(0,m*Ts,Ts)

EEG_df = EEG.to_data_frame()
EEG_df
# Nelle righe abbiamo i campioni, nelle colonne i canali

plt.plot(EEG_df['time'],EEG_df['FP1'])
plt.xlabel('Time[s]')